# 决策树

本文主要参考：

1. [维基百科](https://zh.wikipedia.org/wiki/%E5%86%B3%E7%AD%96%E6%A0%91%E5%AD%A6%E4%B9%A0)
2. [数据挖掘十大经典算法--CART: 分类与回归树](https://wizardforcel.gitbooks.io/dm-algo-top10/content/cart.html)
3. [CART算法](https://zhuanlan.zhihu.com/p/32003259)
4. [机器学习之分类与回归树(CART)](https://zhuanlan.zhihu.com/p/36108972)
5. [分类回归树CART(上)](https://www.cnblogs.com/zhangchaoyang/articles/2709922.html)
6. [决策树](https://github.com/Vay-keen/Machine-learning-learning-notes/blob/master/%E5%91%A8%E5%BF%97%E5%8D%8E%E3%80%8AMachine%20Learning%E3%80%8B%E5%AD%A6%E4%B9%A0%E7%AC%94%E8%AE%B0(5)--%E5%86%B3%E7%AD%96%E6%A0%91.md)

水文上也常常用到各类分类算法，比如统计学，数据挖掘和机器学习中的决策树训练。它使用决策树作为预测模型来预测样本的类标。这种决策树也称作分类树或回归树。在这些树的结构里， 叶子节点给出类标而内部节点代表某个属性。

在数据挖掘中，决策树主要有两种类型:

- 分类树 的输出是样本的类标。 
- 回归树 的输出是一个实数 (例如房子的价格，病人呆在医院的时间等)。

通常决策树主要有三种实现，分别是ID3算法，CART算法和C4.5算法。 本文以CART为例记录。

## 基本概念

决策树是基于树结构来进行决策的，网上看到一个例子十分有趣。母亲想要给自己的女娃介绍一个男朋友，于是有了下面的对话：

  女儿：多大年纪了？
  母亲：26。
  女儿：长的帅不帅？
  母亲：挺帅的。
  女儿：收入高不？
  母亲：不算很高，中等情况。
  女儿：是公务员不？
  母亲：是，在税务局上班呢。
  女儿：那好，我去见见。

该过程就是一个典型的决策树，即相当于**通过年龄、长相、收入和是否公务员**将男童鞋分为**两个类别**：见和不见。
![example](68747470733a2f2f692e6c6f6c692e6e65742f323031382f31302f31372f356263373238656338346137372e706e67.png)
决策过程的每次判定都是对某一属性的“测试”。那么也可知决策树包含：一个根节点、若干个内部节点和若干个叶子节点，易知：

* **每个非叶节点表示一个特征属性**测试。
* 每个分支代表这个特征属性在某个值域上的输出。
* **每个叶子节点存放一个类别**。
* 每个节点包含的样本集合通过属性测试被划分到子节点中，根节点包含样本全集。

术语分类和回归树Classification And Regression Tree (CART) 是决策树的一种实现。CART算法是一种二分递归分割技术，把当前样本划分为两个子样本，使得生成的每个非叶子结点都有两个分支，因此CART算法生成的决策树是结构简洁的二叉树。由于CART算法构成的是一个二叉树，它在每一步的决策时只能是“是”或者“否”，即非叶子节点的特征取值为True和False，左分支取值为True，右分支取值为False，即使一个feature有多个取值，也是把数据分为两部分。CART可以处理连续型变量和离散型变量，利用训练数据递归的划分特征空间进行建树，用验证数据进行剪枝。

- 如果待预测分类是离散型数据，则CART生成分类决策树。
- 如果待预测分类是连续性数据，则CART生成回归决策树。

在CART算法中主要分为两个步骤

1. 将样本递归划分进行建树过程
2. 用验证数据进行剪枝

## 构造

注意构造决策树的输入有两个集合，一个是训练集，一个是属性集。结合前面的例子，就是一个女生已经相亲过有很多男生，这些男生就是训练集，而属性集是女生自己定的条件。构造决策树的动机可以理解成现在有一个新的男生，他想根据这个女生之前对男生的分类来判断自己有没有可能在她的yes中。那现在这个男生就需要构造一个决策树来分析了。决策树的构造是一个递归的过程，如图所示：
![train](68747470733a2f2f692e6c6f6c692e6e65742f323031382f31302f31372f356263373238656363323766652e706e67.png)

结合例子，就是之前全部男生都被女生排除了，那就是全部都是no；如果女生根本就没有选，只是不想相亲，那么就没有属性，全部no，那就只能将所有哥们直接标记成no了；接着是比较正常的情况，对这个女生分析，要分析哪个是最优划分属性，收入还是外貌等等，然后对这个属性的每个取值，生成一个分支，选出各个男生里面符合这个条件的，如果没有符合这个条件的，比如年薪百万的没有，那这些哥们被no的原因就是这个了，全部分到no就完事了；如果还真有，那这个就是yes了，那可能有其他原因被no，就对这几个百万哥继续分，用除了年薪这个属性之外的属性去分，递归重复类似的过程，一直到所有哥们都被分完。

那么可以想象，为什么是先考虑收入，而不是才华呢？你怎么就知道那个妹子不是喜欢有才华的呢？也就是说构造决策树的依据是什么呢？而这就是决策树学习的关键，关键在于**如何选择划分属性**，**不同的划分属性得出不同的分支结构**，从而影响整颗决策树的性能。属性划分的目标是让各个划分出来的**子节点尽可能地“纯”，即属于同一类别**。

接下来就以CART算法为例稍微叙述下。CART算法构建决策树时在每一步选择一个最好的属性来分裂。CART决策树使用**“基尼指数”（Gini index）**来选择划分属性，基尼指数反映的是从样本集D中随机抽取两个样本，其类别标记不一致的概率，因此Gini(D)越小越好。
$$Gini(D)=\sum _{k=1}^{|y|}\sum _{k'\neq k}p_k p_{k'}=1-\sum _{k=1}^{|y|}p_k^2$$
使用某个属性a划分后的基尼指数为：
$$Gini\_index(D,a)=\sum _{v=1}^{V}\frac{|D^v|}{|D|}Gini(D^v)$$
选择基尼指数最小的划分属性。

## 剪枝处理

从决策树的构造流程中我们可以直观地看出：不管怎么样的训练集，决策树总是能很好地将各个类别分离开来，这时就会遇到之前提到过的问题：过拟合（overfitting），即太依赖于训练样本。剪枝（pruning）则是决策树算法对付过拟合的主要手段。这里暂不赘述了。

## 连续值与缺失值处理

对于连续值的属性，每个取值作为一个分支显然不可行，因此需要进行离散化处理，常用的方法为二分法，基本思想为：给定样本集D与连续属性α，二分法试图找到一个划分点t将样本集D在属性α上分为≤t与＞t。

* 首先将α的所有取值按升序排列，所有相邻属性的均值作为候选划分点（n-1个，n为α所有的取值数目）。
* 计算每一个划分点划分集合D（即划分为两个分支）后的信息增益。
* 选择最大信息增益的划分点作为最优划分点。

$$Gain(D,a)=max_{t\in T_a}\ Gain(D,a,t)=max_{t\in T_a}\ Ent(D)-\sum _{\lambda \in {-,+}}\frac{|D_t^v|}{|D|}Ent(D_t^{\lambda})$$

In [1]:
from sklearn import tree
X = [[0, 0], [1, 1]]
Y = [0, 1]
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, Y)
clf.predict([[2., 2.]])

array([1])